In [1]:
from web3 import Web3

web3 = Web3(Web3.HTTPProvider('http://localhost:7545')) # Ganache
web3.isConnected()

True

In [2]:
import json

PATH_TRUFFLE_WK = '/Users/mettinger/Desktop/truffle/lotteries'
truffleFile = json.load(open(PATH_TRUFFLE_WK + '/build/contracts/SimpleLottery.json'))

abi = truffleFile['abi']
bytecode = truffleFile['bytecode']

In [20]:
address1 = '0x8a38346Df3E1F5053e669F9B5272693f90d55876'
address2 = '0x95C39d607610DcB7CB8Ed38502cB073e4591ad59'

In [14]:
address1 = Web3.toChecksumAddress(address1)
ether_balance = web3.fromWei(web3.eth.getBalance(address1),'ether')
ether_balance

Decimal('99.9246718')

In [4]:
contractAddress = '0xd4d8e6a5999d1d8ee0797db0dbd3db5d27653cd1'
lotteryContractAddress = Web3.toChecksumAddress(contractAddress)

contract = web3.eth.contract(address = lotteryContractAddress, abi = abi)

In [5]:
contract.functions.TICKET_PRICE().call()

10000000000000000

In [6]:
contract.functions.ticketingCloses().call()

1544300618

In [21]:
web3.eth.sendTransaction({'from': address2, 'to': lotteryContractAddress, 'value':Web3.toWei(.01,'ether')})

HexBytes('0x41a6eaf5f99fa1abc3a624624d30c558a7cb69885298128d627d27dc4d868894')

In [23]:
contract.functions.tickets(1).call()

'0x95C39d607610DcB7CB8Ed38502cB073e4591ad59'

In [40]:
contract.functions.drawWinner().transact({'from':address1})

HexBytes('0x2623c82c6b20feadae920faf857fbb18bdbc5943b1558686dc24fdfb9f0ecbb4')

In [41]:
contract.functions.winner().call()

'0x8a38346Df3E1F5053e669F9B5272693f90d55876'

In [42]:
ether_balance = web3.fromWei(web3.eth.getBalance(lotteryContractAddress),'ether')
ether_balance

Decimal('0.02')

In [43]:
contract.functions.withdraw().transact({'from':address1})

HexBytes('0xda7278964b7c64068384a07b76622a7c3943de18a9bc3c4644f899a8fde29e76')

In [45]:
ether_balance = web3.fromWei(web3.eth.getBalance(lotteryContractAddress),'ether')
ether_balance

0

In [ ]:
'''
pragma solidity ^0.4.18; 

contract SimpleLottery {
    uint public constant TICKET_PRICE = 1e16; // 0.01 ether

    address[] public tickets;
    address public winner;
    uint public ticketingCloses;

    function SimpleLottery (uint duration) public {
        ticketingCloses = now + duration;
    }

    function buy () public payable {
        require(msg.value == TICKET_PRICE); 
        //require(now < ticketingCloses);

        tickets.push(msg.sender);
    }

    function drawWinner () public {
        require(now > ticketingCloses + 1 minutes);
        require(winner == address(0));

        bytes32 rand = keccak256(
            block.blockhash(block.number-1)
        );
        winner = tickets[uint(rand) % tickets.length];
    }


    function withdraw () public {
        require(msg.sender == winner);
        msg.sender.transfer(this.balance);
    }

    function () payable public {
        buy();
    }
}
'''